In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import IPython
import IPython.display
import matplotlib.pyplot as plt
import sys
from IPython.display import clear_output

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

2023-07-31 01:12:33.839472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 01:12:34.499665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Read in all the files

In [2]:
path = './'
from Window_Generator_package import Window_Generator

# stock_example = pd.read_csv(path + 'Data/Grouped/Consumer and Service Sectors/ALL.AX.csv')
def read_stock(path, file_name):
    stock = pd.read_csv(path + file_name, header = [0])
    stock_index = pd.to_datetime(stock['Date'])
    stock.index = stock_index
    stock = stock.drop('Date', axis = 1)
    return stock
group = ['Consumer and Service Sectors', 'Financial, Healthcare, Technology, and Utilities Sectors', 'Industrial and Infrastructure Sectors']
index = [0, 1, 2]
def read_all(the_path):
    All_df = []
    files = [f for f in os.listdir(the_path) if f.endswith('.csv')]

    for name in files:
        stock = read_stock(the_path + '/', name)
        All_df.append(stock)
    return All_df, files
All_df_0, files = read_all(path + 'Data/Grouped/' + group[index[0]])
stock_example = All_df_0[0]

In [3]:
print(stock_example.shape)
train_series = stock_example[:1008].squeeze()
test_series = stock_example[1008:].squeeze()

(1260, 1)


In [4]:
w = Window_Generator(train_series = train_series, test_series = test_series)
_, _, train_mean, train_std, test_mean, test_std = w.standardization(show=True)
_, _, _, _ = w.window_generation(show = True)
X_train, X_test, Y_train, Y_test = w.data_getter()
train_denormalized = w.denormalize_train()
test_denormalized = w.denormalize_test()

train_series_shape: (1008,)
test_series_shape: (252,)
X_train: (1003, 5, 1)
Y_train: (1003, 1, 1)
X_test: (252, 5, 1)
Y_test: (252, 1, 1)


In [5]:
MAX_EPOCHS = 30

def compile_and_fit(model, X_train, Y_train, patience=5):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(X_train, Y_train, epochs=MAX_EPOCHS, verbose = 0)
    return history

In [6]:
def cnn_model(window_length):
    cnn = Sequential()
    cnn.add(Conv1D(filters=32, kernel_size=window_length, activation='relu', input_shape=(5, 1)))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(64, activation='relu'))
    cnn.add(Dense(1, activation = 'linear'))
    return cnn

cnn = cnn_model(window_length = 5)

2023-07-31 01:13:06.416769: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-31 01:13:06.542540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-31 01:13:06.543016: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-31 01:13:06.545255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-31 01:13:06.545627: I tensorflow/compile

In [ ]:
cnn.summary()

In [ ]:
history = compile_and_fit(cnn, X_train = X_train, Y_train = Y_train)
IPython.display.clear_output()
train_performance = cnn.evaluate(X_train, Y_train)
test_performance = cnn.evaluate(X_test, Y_test)
print('Train:', str(train_performance))
print('Test:', str(test_performance))

In [ ]:
Y_predict = cnn.predict(X_test)
Y_predict = Y_predict.reshape(len(Y_predict))

## As Neural Network doesn't provide the same result each run time, run 5 times and get the average of the prediction

In [ ]:
def obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = 5):
    # The input train and test are all 3D list
    predicted_train_list = []
    predicted_test_list = []
    train_MAE_list = []
    test_MAE_list = []
    for i in range(5):
        cnn = cnn_model(window_length)
        history = compile_and_fit(cnn, X_train = X_train, Y_train = Y_train)
        # IPython.display.clear_output()
        train_performance = cnn.evaluate(X_train, Y_train)[1]
        test_performance = cnn.evaluate(X_test, Y_test)[1]
        # print('Train:', str(train_performance))
        # print('Test:', str(test_performance))
        Y_predict_train = cnn.predict(X_train)
        Y_predict_train = Y_predict_train.reshape(len(Y_predict_train))
        predicted_train_list.append(Y_predict_train)

        Y_predict_test = cnn.predict(X_test)
        Y_predict_test = Y_predict_test.reshape(len(Y_predict_test))
        predicted_test_list.append(Y_predict_test)

        train_MAE_list.append(train_performance)
        test_MAE_list.append(test_performance)

    train_MAE = np.mean(train_MAE_list)
    test_MAE = np.mean(test_MAE_list)

    predicted_train_list = np.array(predicted_train_list)
    predicted_train_list = np.mean(predicted_train_list, axis = 0)
    predicted_test_list = np.array(predicted_test_list)
    predicted_test_list = np.mean(predicted_test_list, axis = 0)
    clear_output(wait=True)

    return predicted_train_list, predicted_test_list, train_MAE, test_MAE


In [ ]:
predicted_train_list, predicted_test_list, train_MAE, test_MAE = obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = 5)

In [ ]:
print(predicted_train_list.shape)
print(predicted_test_list.shape)
print(train_MAE)
print(test_MAE)

In [ ]:
train_index = w.train_index[w.window_length:]
test_index = w.test_index

Y_predict_train = pd.Series(predicted_train_list, index = train_index)
Y_predict_train = Y_predict_train * train_std[w.window_length:] + train_mean[w.window_length:]
Y_actual_train = Y_train.reshape(len(Y_train)) * train_std[w.window_length:] + train_mean[w.window_length:]
Y_predict_train = pd.DataFrame(Y_predict_train, index = train_index)
Y_actual_train = pd.DataFrame(Y_actual_train, index = train_index)

Y_predict_test = pd.Series(predicted_test_list, index = test_index)
Y_predict_test = Y_predict_test * test_std + test_mean
Y_actual_test = Y_test.reshape(len(Y_test)) * test_std + test_mean
Y_predict_test = pd.DataFrame(Y_predict_test, index = test_index)
Y_actual_test = pd.DataFrame(Y_actual_test, index = test_index)


plt.plot(Y_actual_train, label = 'Train Actual')
plt.plot(Y_predict_train, label = 'Train predict')
plt.plot(Y_actual_test, label = 'Test Actual')
plt.plot(Y_predict_test, label = 'Test predict')
plt.legend()
plt.show()

### Store the obtained csv file into Google Drive

In [ ]:
store_path = './Data/Prediction/CNN/'
if os.path.exists(store_path):
    print("The path exists.")
else:
    print("The path doesn't exist.")

## All the code above are examples, here is the training for all stocks

In [ ]:
# run_through all the files inside All_df
def obtain_all(the_index, window_length = 5, show = None):
  group_name = group[index[the_index]]
  All_df, files = read_all(path + 'Data/Grouped/' + group_name)

  total_length = len(All_df)
  train_All_MAE = []
  test_All_MAE = []

  for i in range(total_length):
    # Obtain the Series
    stock = All_df[i]
    file_name = files[i]
    train_series = stock[:1008].squeeze()
    test_series = stock[1008:].squeeze()

    # Data Preparation
    w = Window_Generator(train_series = train_series, test_series = test_series)
    _, _, train_mean, train_std, test_mean, test_std = w.standardization(show=True)
    _, _, _, _ = w.window_generation(show = True)
    X_train, X_test, Y_train, Y_test = w.data_getter()
    train_denormalized = w.denormalize_train()
    test_denormalized = w.denormalize_test()

    # Training and Prediction
    predicted_train_list, predicted_test_list, train_MAE, test_MAE = obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = window_length)
    # print('Train_MAE:', train_MAE)
    # print('Test_MAE:', test_MAE)
    train_All_MAE.append(train_MAE)
    test_All_MAE.append(test_MAE)

    # Prediction Generation (Are train_set)

    train_index = w.train_index[w.window_length:]
    test_index = w.test_index

    Y_predict_train = pd.Series(predicted_train_list, index = train_index)
    Y_predict_train = Y_predict_train * train_std[w.window_length:] + train_mean[w.window_length:]
    Y_actual_train = Y_train.reshape(len(Y_train)) * train_std[w.window_length:] + train_mean[w.window_length:]
    Y_predict_train = pd.DataFrame(Y_predict_train, index = train_index)
    Y_actual_train = pd.DataFrame(Y_actual_train, index = train_index)

    Y_predict_test = pd.Series(predicted_test_list, index = test_index)
    Y_predict_test = Y_predict_test * test_std + test_mean
    Y_actual_test = Y_test.reshape(len(Y_test)) * test_std + test_mean
    Y_predict_test = pd.DataFrame(Y_predict_test, index = test_index)
    Y_actual_test = pd.DataFrame(Y_actual_test, index = test_index)

    # if (show != None):
    #   plt.plot(Y_actual_train, label = 'Train Actual')
    #   plt.plot(Y_predict_train, label = 'Train predict')
    #   plt.plot(Y_actual_test, label = 'Test Actual')
    #   plt.plot(Y_predict_test, label = 'Test predict')
    #   plt.legend()
    #   plt.show()

    # Store the results
    Y_result = pd.concat([pd.concat([Y_actual_train, Y_predict_train], axis = 1), pd.concat([Y_actual_test, Y_predict_test], axis = 1)], axis = 0)
    Y_result.column = ['Actual, Predict']
    Y_result.to_csv(store_path + group_name + '/' + file_name)
    clear_output(wait=True)
    

  return train_All_MAE, test_All_MAE


In [ ]:
train_MAE_1, test_MAE_1 = obtain_all(0, window_length = 5, show = True)

In [ ]:
train_MAE_2, test_MAE_2 = obtain_all(1, window_length = 5, show = True)

In [ ]:
train_MAE_3, test_MAE_3 = obtain_all(2, window_length = 5, show = True)

In [ ]:
train_1 = np.column_stack((train_MAE_1, test_MAE_1))
train_1 = pd.DataFrame(train_1, columns = ['train', 'test'])
train_2 = np.column_stack((train_MAE_2, test_MAE_2))
train_2 = pd.DataFrame(train_2, columns = ['train', 'test'])
train_3 = np.column_stack((train_MAE_3, test_MAE_3))
train_3 = pd.DataFrame(train_3, columns = ['train', 'test'])

In [ ]:
train_1.to_csv(store_path + 'Evaluation' + '/' + 'MAE_group1.csv')
train_2.to_csv(store_path + 'Evaluation' + '/' + 'MAE_group2.csv')
train_3.to_csv(store_path + 'Evaluation' + '/' + 'MAE_group3.csv')